<a href="https://colab.research.google.com/github/richardOlson/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
# Doing some of the imports here
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [0]:
PATH = os.path.join(os.path.curdir, "WA_Fn-UseC_-Telco-Customer-Churn+(1).csv")
PATH

'.\\WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'

In [0]:
colab_path = "/content/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"

In [7]:
!pip install category-encoders

     |████████████████████████████████| 81kB 2.4MB/s 


In [0]:
df = pd.read_csv(colab_path)

In [5]:
df.shape

(7043, 21)

In [0]:
# Loading the data set
df = pd.read_csv(PATH)
df.shape

(7043, 21)

In [0]:
# Looking for null values
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [0]:
# This means that we are doing a binary classification for this data
df["Churn"].unique()

array(['No', 'Yes'], dtype=object)

In [0]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [0]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [0]:
for col in df.columns.tolist():
    print(f"vals for {col}")
    print(pd.unique(df[col]))
    print("----------------")


vals for customerID
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
----------------
vals for gender
['Female' 'Male']
----------------
vals for SeniorCitizen
[0 1]
----------------
vals for Partner
['Yes' 'No']
----------------
vals for Dependents
['No' 'Yes']
----------------
vals for tenure
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
----------------
vals for PhoneService
['No' 'Yes']
----------------
vals for MultipleLines
['No phone service' 'No' 'Yes']
----------------
vals for InternetService
['DSL' 'Fiber optic' 'No']
----------------
vals for OnlineSecurity
['No' 'Yes' 'No internet service']
----------------
vals for OnlineBackup
['Yes' 'No' 'No internet service']
----------------
vals for DeviceProtection
['No' 'Yes' 'No internet service']
----------------
vals f

In [8]:
# Will be doing some one hot encoding of the values that are strings
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# trying the drop of the dataframe
d = df.copy()

In [0]:
def prep_data(df, return_x_frame=False):
    the_df = df.copy() # copying the data 
     
    # doing some droping of the id
    the_df = the_df.drop(columns=["customerID"], axis=1)

    # now doing going to change the TotalCharges to change it to 
    # a numeric
    the_df["TotalCharges"] = pd.to_numeric(the_df["TotalCharges"], errors="coerce")

    # Will now do dropping of the rows that have null in them in the TotalCharges
    the_df = the_df.dropna()

    # will now split the dataFrame into y and x
    y = the_df["Churn"]
    feature_names = [x for x in the_df.columns.tolist() if x != "Churn"]
    x = the_df[feature_names]
    # doing some ordinal encoding
    # ord = OrdinalEncoder(categories=["gender", "Partner", "Dependents"])
    # create the one hot encoding
    vals_to_encode = ["gender", "Partner", "Dependents", "PhoneService", 
                    "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", 
                    "DeviceProtection", "TechSupport", "StreamingTV", 
                    "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]
    encoder = ce.OneHotEncoder(use_cat_names=True, cols=vals_to_encode)
    
    encoded_data = encoder.fit_transform(x)

    # using a scaler to scale the data
    minScaler = MinMaxScaler()
    scaled_data = minScaler.fit_transform(encoded_data)
    
    frame= pd.DataFrame(data=scaled_data, columns=encoder.get_feature_names())
    if return_x_frame == True:
        return frame, scaled_data, y
    return scaled_data,  y

In [0]:
x, y = prep_data(d)

In [0]:
x.shape, y.shape

((7032, 45), (7032,))

In [0]:
# Doing some imports for the model and tuning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [0]:
# Doing more imports
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam, SGD

In [0]:
# checking what a good size of epochs would be

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam):

    # making it so that we get the optizmizer
    opt = optimizer()


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model)

params = {
    "epochs": [10, 15, 25, 50, 100]
}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=4, verbose=1)
grid.fit(x, y)






Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  2.1min finished
Train on 7032 samples
Epoch 1/25
7032/7032 [==============================] - 1s 127us/sample - loss: 0.4760 - accuracy: 0.7632 - mse: 0.1568
Epoch 2/25
7032/7032 [==============================] - 0s 45us/sample - loss: 0.4306 - accuracy: 0.7951 - mse: 0.1405
Epoch 3/25
7032/7032 [==============================] - 0s 42us/sample - loss: 0.4279 - accuracy: 0.7947 - mse: 0.1395
Epoch 4/25
7032/7032 [==============================] - 0s 44us/sample - loss: 0.4246 - accuracy: 0.7985 - mse: 0.1384
Epoch 5/25
7032/7032 [==============================] - 0s 45us/sample - loss: 0.4225 - accuracy: 0.8023 - mse: 0.1374
Epoch 6/25
7032/7032 [==============================] - 0s 43us/sample - loss: 0.4214 - accuracy: 0.7991 - mse: 0.1373
Epoch 7/25
7032/7032 [==============================] - 0s

GridSearchCV(estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001D1E415E588>,
             n_jobs=4, param_grid={'epochs': [10, 15, 25, 50, 100]}, verbose=1)

In [0]:
# These were the params that we did
print("the best param for the epoch size is: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

the best param for the epoch size is:  {'epochs': 25}
the best score was:  0.8041794657707214


In [0]:
# Doiing the hyperparameter tuning but this time leaving the epoch at 25
# tuning the batch size

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam):

    # making it so that we get the optizmizer
    opt = optimizer()


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [2,8, 16, 32]
}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=4, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  4.9min finished
Train on 7032 samples
Epoch 1/25
7032/7032 [==============================] - 1s 93us/sample - loss: 0.4670 - accuracy: 0.7760 - mse: 0.1531
Epoch 2/25
7032/7032 [==============================] - 0s 49us/sample - loss: 0.4326 - accuracy: 0.7910 - mse: 0.1411
Epoch 3/25
7032/7032 [==============================] - 0s 42us/sample - loss: 0.4259 - accuracy: 0.7964 - mse: 0.1387
Epoch 4/25
7032/7032 [==============================] - 0s 42us/sample - loss: 0.4259 - accuracy: 0.7985 - mse: 0.1388
Epoch 5/25
7032/7032 [==============================] - 0s 41us/sample - loss: 0.4239 - accuracy: 0.7964 - mse: 0.1383
Epoch 6/25
7032/7032 [==============================] - 0s 41us/sample - loss: 0.4207 - accuracy: 0.8006 - mse: 0.1372
Epoch 7/25
7032/7032 [==============================] - 0s 

In [0]:
from tensorflow.keras.optimizers import SGD

In [17]:
# Doiing the hyperparameter tuning but this time leaving the epoch at 25
# and the batch_size of 32
# 
#  Trying a new type of optimizer sgd
# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = SGD):

    # making it so that we get the optizmizer
    opt = optimizer()


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.5s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.5611 - accuracy: 0.7342 - mse: 0.1882
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.5293 - accuracy: 0.7342 - mse: 0.1757
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.5045 - accuracy: 0.7344 - mse: 0.1661
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4821 - accuracy: 0.7408 - mse: 0.1577
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4641 - accuracy: 0.7648 - mse: 0.1512
Epoch 6/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4523 - accuracy: 0.7782 - mse: 0.1473
Epoch 7/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4442 - accuracy: 0.7861 - mse: 0.1446
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4405 - accuracy: 0.7921 - mse: 0.1435
Epoch 9/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4379 - accur

In [0]:
# This shows that ADAm was better, so we will keep adam and now tune the learning rate

In [20]:
# Doiing the hyperparameter tuning but this time leaving the epoch at 25
# and the batch_size of 32
# 
#  Tuning the learning rate of the Adam
# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001, .01, .1, .0001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.0min finished


Epoch 1/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4786 - accuracy: 0.7621 - mse: 0.1580
Epoch 2/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4336 - accuracy: 0.7920 - mse: 0.1414
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4265 - accuracy: 0.7935 - mse: 0.1389
Epoch 4/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4256 - accuracy: 0.7982 - mse: 0.1384
Epoch 5/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4261 - accuracy: 0.7944 - mse: 0.1390
Epoch 6/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8020 - mse: 0.1376
Epoch 7/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4206 - accuracy: 0.7993 - mse: 0.1367
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4234 - accuracy: 0.7981 - mse: 0.1382
Epoch 9/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4206 - accur

In [21]:
# Tuning now if I should change the number of layers
# I am taking out one of the layers to see if the model gets better
# 
# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    #model.add(Dense(64, activation="relu")) 
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001 ]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   29.6s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4700 - accuracy: 0.7713 - mse: 0.1543
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4317 - accuracy: 0.7925 - mse: 0.1408
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4258 - accuracy: 0.7954 - mse: 0.1387
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.8002 - mse: 0.1376
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4209 - accuracy: 0.8025 - mse: 0.1369
Epoch 6/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.8016 - mse: 0.1373
Epoch 7/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4211 - accuracy: 0.8009 - mse: 0.1369
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.8029 - mse: 0.1367
Epoch 9/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4201 - accur

In [0]:
# removing the layer made it only slightly worsened

In [22]:
# Tunning now the activation function
# will be changing the activation function to all relu except at the end

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="relu", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   30.5s finished


Epoch 1/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4449 - accuracy: 0.7840 - mse: 0.1457
Epoch 2/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4209 - accuracy: 0.7991 - mse: 0.1372
Epoch 3/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4171 - accuracy: 0.8033 - mse: 0.1361
Epoch 4/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4130 - accuracy: 0.8036 - mse: 0.1345
Epoch 5/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4079 - accuracy: 0.8082 - mse: 0.1326
Epoch 6/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4071 - accuracy: 0.8062 - mse: 0.1325
Epoch 7/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4054 - accuracy: 0.8094 - mse: 0.1317
Epoch 8/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8107 - mse: 0.1303
Epoch 9/25
220/220 [==============================] - 0s 2ms/step - loss: 0.3991 - accur

In [0]:
# When changing the first layer to a relu layer, it caused the model to worsen from 
# .8014 to .7817

In [24]:
# Will now be making all the activation functions sigmoid

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(128, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="sigmoid"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   30.0s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4927 - accuracy: 0.7585 - mse: 0.1625
Epoch 2/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4314 - accuracy: 0.7961 - mse: 0.1402
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4270 - accuracy: 0.7939 - mse: 0.1388
Epoch 4/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4238 - accuracy: 0.7985 - mse: 0.1379
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4242 - accuracy: 0.8032 - mse: 0.1378
Epoch 6/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4225 - accuracy: 0.7985 - mse: 0.1376
Epoch 7/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4237 - accuracy: 0.7969 - mse: 0.1378
Epoch 8/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4232 - accuracy: 0.7993 - mse: 0.1376
Epoch 9/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4210 - accur

In [0]:
# When making all the activation functions into a sigmoid activation function 
# it cause the model to do also worse at .80007 compared to .8014

In [25]:
# Will try to tune the number of neurons in the layers
# will try to lessen the number of the first hidden layer to 64

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(64, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   28.9s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4814 - accuracy: 0.7602 - mse: 0.1587
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4311 - accuracy: 0.7932 - mse: 0.1403
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4274 - accuracy: 0.7948 - mse: 0.1392
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4233 - accuracy: 0.7991 - mse: 0.1381
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.7978 - mse: 0.1389
Epoch 6/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4225 - accuracy: 0.7971 - mse: 0.1378
Epoch 7/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.7978 - mse: 0.1374
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4201 - accuracy: 0.7989 - mse: 0.1368
Epoch 9/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4203 - accur

In [0]:
# When making the number of nuerons in the first layer to 64 instead of 128. 
# It improved the model to .8030 from .8014


In [26]:
# Will try to lower the amount in the second hidden layer to just 32


# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(64, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   28.2s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.5137 - accuracy: 0.7336 - mse: 0.1714
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4364 - accuracy: 0.7948 - mse: 0.1423
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4278 - accuracy: 0.7958 - mse: 0.1394
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4254 - accuracy: 0.7976 - mse: 0.1385
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4235 - accuracy: 0.7996 - mse: 0.1378
Epoch 6/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.8029 - mse: 0.1377
Epoch 7/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.7979 - mse: 0.1375
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4212 - accuracy: 0.7974 - mse: 0.1373
Epoch 9/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4205 - accur

In [0]:
# Lowering the second hidden layer to just 32 from 64 made the model
# perform just a little worse from .803 to .80006

In [27]:
# Will try to raise the number of neurons in the second layer to 128

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(64, activation="sigmoid", input_shape=(x.shape[1],)))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   29.7s finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4921 - accuracy: 0.7634 - mse: 0.1623
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4339 - accuracy: 0.7910 - mse: 0.1414
Epoch 3/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4272 - accuracy: 0.7966 - mse: 0.1391
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4258 - accuracy: 0.7974 - mse: 0.1388
Epoch 5/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4247 - accuracy: 0.8003 - mse: 0.1384
Epoch 6/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.8002 - mse: 0.1378
Epoch 7/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8025 - mse: 0.1369
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4224 - accuracy: 0.7992 - mse: 0.1377
Epoch 9/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4240 - accur

In [0]:
# When increasing the number of the neurons in the second hidden layer
# it made the model perform just a little worse from .803 to .802

In [0]:
from tensorflow.keras.layers import  Dropout

In [29]:
# Tuning using dropout

# Doing a random seed
seed = 42
np.random.seed(seed=seed)

# Creating the function that will do building of model

def build_model(x=x, optimizer = Adam, lr=.001, drop_out_1=True):

    # making it so that we get the optizmizer
    opt = optimizer(learning_rate=lr)


    model = Sequential()
    # add layers
    model.add(Dense(64, activation="sigmoid", input_shape=(x.shape[1],)))
    # Adding the possibility to have a dropout layer
    if drop_out_1 == True:
      model.add(Dropout(.2))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "mse"])
    # returning the model
    return model

    # will now put the model in the kerasclassifier
model = KerasClassifier(build_fn=build_model, verbose=1)

params = {
    "epochs": [25], 
    "batch_size": [32], 
    "lr": [.001], 
    "drop_out_1": [True, False]

}

# now getting the grid search
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, verbose=1)
grid.fit(x, y)


# These were the params that we did
print("the best params for are: ", grid.best_params_)
print("the best score was: ", grid.best_score_)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.0min finished


Epoch 1/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4790 - accuracy: 0.7679 - mse: 0.1580
Epoch 2/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4315 - accuracy: 0.7948 - mse: 0.1406
Epoch 3/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4268 - accuracy: 0.7968 - mse: 0.1390
Epoch 4/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4267 - accuracy: 0.7954 - mse: 0.1391
Epoch 5/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.7991 - mse: 0.1379
Epoch 6/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4238 - accuracy: 0.7991 - mse: 0.1381
Epoch 7/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4229 - accuracy: 0.7995 - mse: 0.1377
Epoch 8/25
220/220 [==============================] - 0s 1ms/step - loss: 0.4223 - accuracy: 0.7996 - mse: 0.1374
Epoch 9/25
220/220 [==============================] - 0s 2ms/step - loss: 0.4209 - accur

In [0]:
# This shows that the dropout when set to False seemed to be the best

In [0]:
# I am using the default initialization mode.


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?